In [1]:
import pandas as pd
import spacy
from spacy.lang.es.stop_words import STOP_WORDS as STOP_WORDS_ES
from spacy.lang.en.stop_words import STOP_WORDS as STOP_WORDS_EN
import string

In [2]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
name_file = "sample2.csv"

In [4]:
# Cargar los datos
file_path = './data/originals/' + name_file
df_all = pd.read_csv(file_path, delimiter=',')

In [5]:
df_all.shape

(494, 61)

In [6]:
df_all.keys()

Index(['NOMBRE', 'IDDECINVPROYECTOPRODTERMIN', 'IDDECINVPROYECTO',
       'IDDECINVTIPOPRODUCTOESPERADO', 'TITULO', 'ANIO', 'VOLUMEN', 'PAGINAS',
       'NOMBREREVISTA', 'ISINDEX', 'BDREVISTA', 'ISSN', 'FACTIMPACTO',
       'EDITORIAL', 'ESTADOPUBLICACION', 'ESPECIFICACION', 'URL',
       'ULTIMO_CAMBIO', 'NEVENTO', 'LEVENTO', 'FEVENTO', 'ADJUNTO',
       'FECHAPUBLICACION', 'RESUMEN', 'KEYS', 'ASIST_OBJETIV',
       'INNOVA_VERIFIC', 'LOGROS_SISTEMAC', 'RETOS_COMUNIC', 'IDUNIDAD',
       'TIPOLAB', 'MARCA', 'MODELO', 'SERIE', 'INVENTARIO', 'ESTADO',
       'FRECUENCIA', 'FVENCIMIENTO', 'SJR', 'CUARTIL', 'IDDECINVREVISTA',
       'APROBADO', 'IDDECINVMOTIVOPRODTERMIN', 'OBSERVACIONARPOB',
       'IDPUBLICACIONPERSONA', 'ADJUNTO2', 'ADJUNTO3', 'ADJUNTO4',
       'CODIGOPRODUCTO', 'TIPOTESIS', 'REVISIONPARES', 'DOI', 'INVENTOR',
       'SUBCLASE', 'URLREVISTA', 'MATERIAINTEGRADORA', 'NUMEROSOLICITUD',
       'OBJGENERAL', 'ESTAELIMINADO', 'REFARCHIVO', 'ENLACE_FINAL'],
      dtype='objec

In [7]:
# df_all.head()

In [8]:
data = df_all[[#'IDDECINVPROYECTOPRODTERMIN', 'IDDECINVPROYECTO', 
                 'IDDECINVTIPOPRODUCTOESPERADO', "TITULO", "ANIO", "VOLUMEN", "PAGINAS", "ISSN", "DOI", "RESUMEN", "KEYS", "ENLACE_FINAL", "CUARTIL", "OBJGENERAL"]]

###### Eliminar filas resumen vacio

In [9]:
data = data.dropna(subset=['RESUMEN'])

##### Filtro de repetidos y nulos para DOI, CUARTIL Y PAGINAS

In [10]:
# Función mejorada para verificar si PAGINAS es numérica y menor a 100
def es_pagina_valida(x):
    try:
        # Convertir a numérico y verificar si es menor a 100
        return pd.to_numeric(x, errors='coerce') < 100
    except:
        return False

In [11]:
# Mostrar el tipo de dato de la columna PAGINAS
print("Tipo de dato de la columna PAGINAS:")
print(data['PAGINAS'].dtype)
# print("\nValores de PAGINAS:")
# print(data['PAGINAS'])

# # Probar la función con cada valor
# print("\nPrueba de validación de páginas:")
# for pagina in data['PAGINAS']:
#     print(f"Página: {pagina}, Es válida?: {es_pagina_valida(pagina)}")

Tipo de dato de la columna PAGINAS:
object


In [12]:
# Crear columnas auxiliares para el ordenamiento
data['doi_valido'] = data['DOI'].notna()
data['cuartil_valido'] = data['CUARTIL'].notna()
data['paginas_validas'] = data['PAGINAS'].apply(es_pagina_valida)

# Ordenar el DataFrame por las condiciones
df_sorted = data.sort_values(
    by=['TITULO', 'doi_valido', 'cuartil_valido', 'paginas_validas'],
    ascending=[True, False, False, False]
)

In [13]:
# data[~data['cuartil_valido']]

In [14]:
# Eliminar duplicados manteniendo la primera ocurrencia
df_final = df_sorted.drop_duplicates(subset=['TITULO'], keep='first')

# Eliminar columnas auxiliares
df_final = df_final.drop(['doi_valido', 'cuartil_valido', 'paginas_validas'], axis=1)

In [15]:
print("DataFrame final después de eliminar duplicados con las condiciones especificadas:")
df_final.shape, data.shape

DataFrame final después de eliminar duplicados con las condiciones especificadas:


((443, 12), (467, 15))

In [16]:
data = df_final.copy()

##### Cuantificación de variables

###### CUARTIL

In [17]:
# Función para convertir cuartiles a números
def convertir_cuartil(cuartil):
    if pd.isna(cuartil):
        return 0
    # Usando un diccionario para el mapeo
    cuartil_map = {'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4, "SQ": 0}
    return cuartil_map.get(cuartil, 5)

In [18]:
# from sklearn.preprocessing import LabelEncoder

# object_cols = ["CUARTIL"]
# #Utilizamos el Label Encoding 
# LE=LabelEncoder()
# for i in object_cols:
#     data[i]=data[[i]].apply(LE.fit_transform)
    
# print("Ahora las características son numéricas")
# data

In [19]:
data['CUARTIL'] = data['CUARTIL'].apply(convertir_cuartil)

In [20]:
data['CUARTIL'].unique()

array([0, 1, 3, 2, 4], dtype=int64)

###### PAGINAS

In [21]:
def extraer_paginas(texto):
    limit_page = 150

    if pd.isna(texto):
        return None
    
    texto = str(texto).lower().strip()
    
    # Casos a descartar
    if texto in ['nn', 's/n', 'online']:
        return None
        
    # Caso: "091002 (17 pages)" - extraer número entre paréntesis
    if 'pages)' in texto:
        import re
        match = re.search(r'\((\d+)\s*pages\)', texto)
        if match:
            return int(match.group(1))
    
    # Caso: "p. 8843." - extraer número después de p. o pp.
    if texto.startswith('p.') or texto.startswith('pp.'):
        import re
        numeros = re.findall(r'\d+', texto)
        if numeros:
            num = int(numeros[0])
            return num if num < limit_page else None
    
    # Caso: rango de páginas (e.g., "1026-1042")
    if '-' in texto:
        try:
            inicio, fin = map(int, texto.replace('pp.', '').replace('p.', '').strip().split('-'))
            if fin > inicio and (fin - inicio) < limit_page:  # verificación razonable
                return fin - inicio + 1
        except:
            return None
    
    # Caso: número simple
    try:
        import re
        numeros = re.findall(r'\d+', texto)
        if numeros:
            num = int(numeros[0])
            return num if num < limit_page else None
    except:
        return None
    
    return None

In [22]:
# Aplicar la función y convertir a tipo Int64
data['PAGINAS'] = pd.Series(data['PAGINAS'].apply(extraer_paginas), dtype='Int64')

In [23]:
data['PAGINAS'].unique()

<IntegerArray>
[  11, <NA>,   16,   15,   18,    8,    7,   10,    6,    5,   17,   21,    3,
   14,   29,   12,    4,   20,    9,   33,    2,   25,   30,    0,   19,    1,
   31,   32,   27,   22,   34,   69,   13,   37,   97,   35,   26,   53,   24,
   23,   39,   42,   44,   49,   28]
Length: 45, dtype: Int64

In [24]:
# data[data['PAGINAS_NUM'] == 0]

In [25]:
# de la columna PAGINAS_NUM descartar los 0 y los nulos
print(data.shape)
data = data[(~data["PAGINAS"].isna())] #& data["PAGINAS_NUM"] > 0)]
print(data.shape) # 340 o 148

(443, 12)
(333, 12)


In [26]:
data['PAGINAS'].unique()

<IntegerArray>
[11, 16, 15, 18,  8,  7, 10,  6,  5, 17, 21,  3, 14, 29, 12,  4, 20,  9, 33,
  2, 25, 30,  0, 19,  1, 31, 32, 27, 22, 34, 69, 13, 37, 97, 35, 26, 53, 24,
 23, 39, 42, 44, 49, 28]
Length: 44, dtype: Int64

##### Preprocesamiento de textos 
Lematizacion y union de las variables TITULO, RESUMEN, KEYS

In [ ]:
# # modelo en español de spaCy
# !python -m spacy download es_core_news_sm

In [ ]:
# Cargar modelo spaCy español
nlp = spacy.load("es_core_news_sm")
# Stopwords en español y en inglés
stop_words = set(STOP_WORDS_ES) | set(STOP_WORDS_EN)


# from nltk.corpus import stopwords
# nltk.download('stopwords')
# stopwords_es = set(stopwords.words('spanish'))
# stopwords_en = set(stopwords.words('english'))
# stop_words = stopwords_es | stopwords_en

In [ ]:
# Función de limpieza y lematización
def procesar_texto(texto):
    if pd.isna(texto):
        return ""
    # Unicode, minúsculas, quitar puntuación
    texto = texto.lower().translate(str.maketrans('', '', string.punctuation))
    # Procesar con spaCy
    doc = nlp(texto)

    # Lematizar y quitar stop_words
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and not token.is_punct and not token.is_space]
    return " ".join(tokens)

# Unir columnas y procesar
def crear_corpus(row):
    texto = " ".join([str(row['TITULO']), str(row['RESUMEN']), str(row['KEYS'])])
    return procesar_texto(texto)

In [29]:
# Crear campo corpus
data['CORPUS'] = data.apply(crear_corpus, axis=1)

# print(data[['corpus']])

##### Eliminacionde columnas innecesarias

In [30]:
lista_de_columnas = ['TITULO', 'RESUMEN', 'KEYS', 'OBJGENERAL', 'ISSN', 'ENLACE_FINAL']
# OBJGENERAL todo es nulo actualmente 
# ISSN hay algunos vacios
data.drop(columns=lista_de_columnas, axis=1, inplace=True)

In [31]:
data.head()

,IDDECINVTIPOPRODUCTOESPERADO,ANIO,VOLUMEN,PAGINAS,DOI,CUARTIL,CORPUS
287,25,2025,23,11,revistas.espol.edu.ec/index.php/matematica,0,generalization zariouh’s property gaz local sp...
376,24,2025,25,16,10.1186/s12870-025-06196-4,1,exploring benefits amf colonization improving ...
375,24,2025,15,15,I 10.3389/fpls.2024.1500894,1,inoculation micromonospora sp enhances carbohy...
377,24,2024,24,18,10.1186/s12870-024-05423-8,1,mitigating cold stress rice study genotype per...
386,24,2025,2025,8,10.1155/ijmm/5191108,3,characterization affinir primal topological sp...


In [32]:
data.keys()

Index(['IDDECINVTIPOPRODUCTOESPERADO', 'ANIO', 'VOLUMEN', 'PAGINAS', 'DOI',
       'CUARTIL', 'CORPUS'],
      dtype='object')

##### View data

In [33]:
data.shape, df_all.shape

((333, 7), (494, 61))

In [34]:
data.describe()

,IDDECINVTIPOPRODUCTOESPERADO,ANIO,VOLUMEN,PAGINAS,CUARTIL
count,333.000000,333.000000,3.330000e+02,333.0,333.000000
mean,30.207207,2023.660661,7.706979e+03,13.15015,1.288288
std,9.788225,1.438137,1.370001e+05,10.154305,1.292150
min,24.000000,2016.000000,0.000000e+00,0.0,0.000000
25%,24.000000,2024.000000,1.000000e+00,7.0,0.000000
50%,24.000000,2024.000000,1.400000e+01,11.0,1.000000
75%,46.000000,2024.000000,5.100000e+01,17.0,2.000000
max,46.000000,2025.000000,2.500196e+06,97.0,4.000000


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333 entries, 287 to 68
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   IDDECINVTIPOPRODUCTOESPERADO  333 non-null    int64 
 1   ANIO                          333 non-null    int64 
 2   VOLUMEN                       333 non-null    int64 
 3   PAGINAS                       333 non-null    Int64 
 4   DOI                           333 non-null    object
 5   CUARTIL                       333 non-null    int64 
 6   CORPUS                        333 non-null    object
dtypes: Int64(1), int64(4), object(2)
memory usage: 21.1+ KB


In [36]:
data.isna().sum()

IDDECINVTIPOPRODUCTOESPERADO    0
ANIO                            0
VOLUMEN                         0
PAGINAS                         0
DOI                             0
CUARTIL                         0
CORPUS                          0
dtype: int64

In [37]:
data.head()

,IDDECINVTIPOPRODUCTOESPERADO,ANIO,VOLUMEN,PAGINAS,DOI,CUARTIL,CORPUS
287,25,2025,23,11,revistas.espol.edu.ec/index.php/matematica,0,generalization zariouh’s property gaz local sp...
376,24,2025,25,16,10.1186/s12870-025-06196-4,1,exploring benefits amf colonization improving ...
375,24,2025,15,15,I 10.3389/fpls.2024.1500894,1,inoculation micromonospora sp enhances carbohy...
377,24,2024,24,18,10.1186/s12870-024-05423-8,1,mitigating cold stress rice study genotype per...
386,24,2025,2025,8,10.1155/ijmm/5191108,3,characterization affinir primal topological sp...


In [38]:
data.to_csv("data/cuantifications/" + name_file, index=False)